# fine-tuning

In [3]:
import datetime

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import plot_model
from keras import backend as K

'''
設定
'''

## 学習のハイパーパラメータ
batch_size = 128
n_class = 5
epochs = 5

## 画像の入力サイズ
img_rows, img_cols = 28, 28
## CNNのハイパーパラメータ
filters = 32
pool_size = 2
kernel_size = 3

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

# データセット

In [5]:
'''
ソース、ターゲットデータ作成
'''
## MNISTのデータセットを取得する．
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## データセットを [0..4] と [5..9] に分割する．
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

# ネットワークの構造

In [8]:
'''
ネットワーク構造の定義
'''
## 特徴抽出層 (feature_layers) の定義
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

## 識別層 (classification_layers) の定義
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(n_class),
    Activation('softmax')
]

## 畳み込み層 + 全連結層 のモデルを作る．
model = Sequential(feature_layers + classification_layers)

# モデルの学習関数

In [9]:
def train_model(model, train, test, n_class):
    '''
    前処理
    '''
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # one-hot ベクトルに変換
    y_train = keras.utils.to_categorical(train[1], n_class)
    y_test = keras.utils.to_categorical(test[1], n_class)

    '''
    モデルの設定
    '''
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    '''
    トレーニングデータによるモデルの学習
    '''
    t = datetime.datetime.now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time:{}'.format(datetime.datetime.now() - t))

    '''
    テストデータによるモデルの評価
    '''
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])


# ステップ1. ソースモデルの学習

In [10]:
'''
fine-tuning の元となるモデルの学習
'''
## [0..4] の5種類の画像を用いてモデルを学習する．
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), n_class)

W0906 15:45:54.606003 20124 deprecation_wrapper.py:119] From C:\Users\haruk\Anaconda3\envs\ai-gpu\Lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0906 15:45:54.621960 20124 deprecation_wrapper.py:119] From C:\Users\haruk\Anaconda3\envs\ai-gpu\Lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0906 15:45:54.687783 20124 deprecation.py:323] From C:\Users\haruk\Anaconda3\envs\ai-gpu\Lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 7s 220us/step - loss: 0.1358 - acc: 0.9586 - val_loss: 0.0190 - val_acc: 0.9934
Epoch 2/5
30596/30596 [==============================] - 3s 97us/step - loss: 0.0365 - acc: 0.9887 - val_loss: 0.0119 - val_acc: 0.9953
Epoch 3/5
30596/30596 [==============================] - 3s 98us/step - loss: 0.0234 - acc: 0.9930 - val_loss: 0.0081 - val_acc: 0.9975
Epoch 4/5
30596/30596 [==============================] - 3s 97us/step - loss: 0.0196 - acc: 0.9939 - val_loss: 0.0069 - val_acc: 0.9973
Epoch 5/5
30596/30596 [==============================] - 3s 97us/step - loss: 0.0150 - acc: 0.9951 - val_loss: 0.0046 - val_acc: 0.9984
Training time:0:00:18.894578
Test score: 0.004625545279065406
Test accuracy: 0.9984432769021211


# ステップ2. 重み固定の範囲を決定

In [11]:
'''
fine-tuning
'''
## 畳み込み層を固定する．
for l in feature_layers:
    l.trainable = False

# ステップ3. ターゲットモデルの学習

In [12]:
## [5..9] の数字データを用いてモデルを学習する．
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), n_class)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 2s 65us/step - loss: 0.3635 - acc: 0.9068 - val_loss: 0.0562 - val_acc: 0.9817
Epoch 2/5
29404/29404 [==============================] - 2s 56us/step - loss: 0.0833 - acc: 0.9753 - val_loss: 0.0346 - val_acc: 0.9885
Epoch 3/5
29404/29404 [==============================] - 2s 55us/step - loss: 0.0624 - acc: 0.9810 - val_loss: 0.0291 - val_acc: 0.9914
Epoch 4/5
29404/29404 [==============================] - 2s 55us/step - loss: 0.0471 - acc: 0.9859 - val_loss: 0.0284 - val_acc: 0.9909
Epoch 5/5
29404/29404 [==============================] - 2s 54us/step - loss: 0.0425 - acc: 0.9862 - val_loss: 0.0235 - val_acc: 0.9922
Training time:0:00:08.553231
Test score: 0.023548187684005566
Test accuracy: 0.992182678461222


In [ ]:
## ネットワークの構造を画像で出力する．
plot_model(model,
           to_file='./result/model.png',
           show_shapes=True)